In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
import pymongo
import tweepy
import requests
import os
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs
from requests import get
from dotenv import load_dotenv
load_dotenv()

In [2]:
def selget(url):
    chromedriver = "chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    time.sleep(5)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    driver.close()
    return(soup)

In [3]:
def splintget(url):
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(url)
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = bs(html, 'html.parser')
    browser.quit()
    return(soup)
    # Retrieve the featured image element


In [4]:
#Task 1 - Get Mars headlines from this website
url = 'https://mars.nasa.gov/news/'

In [5]:
#Call the selget procedure becuase it loads too fast otherwise. passes url, parses it via beautifulsoup and returned to 
#variable soup as a soup object
soup = selget(url)

In [6]:
# Preping the variables as list for the Headlines and Paragraphs
news_tease = soup.find('div', class_='article_teaser_body').text
news_title = soup.find('div', class_='list_text').find('a')
news_title = news_title.text.strip()

In [7]:
print(news_title)
print(news_tease)

NASA's Perseverance Mars Rover Gets Balanced
The mission team performed a crucial weight-balancing test on the rover in preparation for this summer's history-making launch to the Red Planet.


In [8]:
# Task using splinter to get the featured image from this website as defined in the url variable
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [9]:
#browser window should close when done, soup ojbect returned.
soup = splintget(url)

In [10]:
#let's start digging. and cleaning up the data scraped.
imgElement = soup.find('article', class_='carousel_item')['style']
imgElement = imgElement.split("'",1)[1]
imgElement = imgElement.rsplit("'",1)

featured_image_url = "https://www.jpl.nasa.gov" + imgElement[0]

In [11]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19913-1920x1200.jpg'

# TWITTER (MARS WEATHER)

In [12]:
consumer_key = os.environ.get('twitter_key')
consumer_secret = os.environ.get('twitter_secret')
access_token = os.environ.get('twitter_token')
access_token_secret = os.environ.get('twitter_token_secret')
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [13]:

tweetList = []
response = api.user_timeline(screen_name = 'MarsWxReport', count=1)

for result in api.search(q="MarsWxReport"):
    tweetList.append(result.text)

mars_weather = tweetList[0]

print(mars_weather)

RT @MarsWxReport: InSight sol 496 (2020-04-18) low -94.6ºC (-138.4ºF) high -6.2ºC (20.9ºF)
winds from the SW at 4.6 m/s (10.3 mph) gusting…


# MARS FACTS

In [14]:
url = 'https://space-facts.com/mars/'

In [15]:
tables = pd.read_html(url)

In [16]:
df = tables[0]

In [17]:
df.columns = ['', 'Value']

In [18]:
df.set_index('', inplace=True)

In [19]:
df_html = df.to_html()

In [20]:
print(df_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Mars Hemispheres

In [21]:
titles = []
links = []

In [22]:
def retrieve(url):
    soup = selget(url)
    content = soup.find(id="wide-image")
    imageContent = content.find('a')['href']
    title1 = soup.title.text.strip()
    title1 = title1.split('|')[0]
    titles.append(title1)
    links.append(imageContent)
    

In [23]:
def getlinks(url):
    soup = selget(url)
    content = soup.find(id="product-section")
    bodyContent = content.find_all("div", class_="description")
    
    for image in bodyContent:
        suburl = image.find("a")['href']  
        fullurl = "https://astrogeology.usgs.gov" + suburl
        #imageloc.append(fullurl)
        retrieve(fullurl)
    




In [24]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
getlinks(url)

In [25]:
hemisphere_image_urls = []
r=0
for r in range(len(titles)):
    p_dict ={}
    p_dict["title"] = titles[r]
    p_dict["img_url"] = links[r]
    hemisphere_image_urls.append(p_dict)
    

In [26]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
